<a href="https://colab.research.google.com/github/ramsoi53/ramsoi/blob/main/Europe_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#라이브러리 Import

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2442 entries, 0 to 2441
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Year         2442 non-null   int64  
 1   Location     2442 non-null   object 
 2   Survey_type  2442 non-null   object 
 3   Surveyor     2442 non-null   object 
 4   qa           2442 non-null   int64  
 5   Maker        2442 non-null   int64  
 6   Address      2442 non-null   object 
 7   City         2442 non-null   object 
 8   Country      2442 non-null   object 
 9   origin       0 non-null      float64
dtypes: float64(1), int64(3), object(6)
memory usage: 190.9+ KB


In [ ]:
# 세 지부별 검사량 변동

pivot_table = data.pivot_table(index='Location', columns='Year', aggfunc='size', fill_value=0)
pivot_table

Year,2021,2022,2023
Location,,,
cph,361,498,391
mil,214,318,351
par,67,82,160


In [ ]:
# Create a pivot table to summarize the workload by location and year, including the 'qa' column
pivot_table_with_qa = data.pivot_table(index='Location', columns='Year', values='qa', aggfunc='sum', fill_value=0)

pivot_table_with_qa

Year,2021,2022,2023
Location,,,
cph,117,104,103
mil,156,191,260
par,0,0,18


In [ ]:
surveyor = df_data['Surveyor'].value_counts()
surveyor

Surveyor
choijongyuel     803
moonkyunghak     575
chohyunchul      421
kimkyongmin      309
kimjinho         307
byunjungkeaun     26
jinjungkwang       1
Name: count, dtype: int64

In [ ]:
# 지부 1인당 평균 검사량

par_surveyor_counts = data[data['Location'] == 'mil']['Surveyor'].value_counts()
par_surveyor_counts

Surveyor
moonkyunghak    575
kimjinho        307
jinjungkwang      1
Name: count, dtype: int64

In [ ]:
#QA전 1인당 평균 검사량
# Filter out surveyors with workload of 10 or less
filtered_data = data.groupby(['Location', 'Surveyor', 'Year']).filter(lambda x: len(x) > 10)

# Calculate the average workload per surveyor per location per year
average_workload = filtered_data.groupby(['Location', 'Year'])['Surveyor'].value_counts().unstack().fillna(0).mean(axis=1).unstack()

average_workload



Year,2021,2022,2023
Location,,,
cph,60.166667,83.000000,65.166667
mil,35.666667,52.666667,58.500000
par,11.166667,13.666667,26.666667


In [ ]:
#QA 후 1인당 평균 검사량

# Recalculate the average workload per surveyor per location per year,
# considering only surveyors with workloads greater than 10 for each year separately.

# Step 1: Filter out entries where qa=1
filtered_data_no_qa = data[data['qa'] == 0]

# Step 2: Calculate the total workload per surveyor per year for each location
workload_per_surveyor_yearly = filtered_data_no_qa.groupby(['Location', 'Surveyor', 'Year']).size().reset_index(name='Workload')

# Step 3: Filter out surveyors with a total workload of 10 or less for each year separately
workload_filtered_yearly = workload_per_surveyor_yearly[workload_per_surveyor_yearly['Workload'] > 10]

# Step 4: Calculate the average workload per surveyor per year for each location
average_workload_per_year_yearly = workload_filtered_yearly.groupby(['Location', 'Year']).agg({'Workload': 'mean'}).unstack()

average_workload_per_year_yearly



Workload              
Year         2021   2022   2023
Location                       
cph         244.0  197.0  144.0
mil          58.0  125.0   45.5
par          67.0   82.0  142.0

In [ ]:
# 23년 mil 검사원 숫자를 1로 재계산
# Adjust the 2023 data for 'mil' to have only one surveyor by summing the workloads
mil_2023_workload_sum = workload_filtered_yearly[(workload_filtered_yearly['Location'] == 'mil') & (workload_filtered_yearly['Year'] == 2023)]['Workload'].sum()

# Create a new DataFrame for the adjusted data
mil_2023_adjusted = pd.DataFrame({'Location': ['mil'], 'Surveyor': ['combined'], 'Year': [2023], 'Workload': [mil_2023_workload_sum]})

# Exclude the original 2023 data for 'mil' and replace with adjusted data
workload_filtered_yearly_adjusted = workload_filtered_yearly[~((workload_filtered_yearly['Location'] == 'mil') & (workload_filtered_yearly['Year'] == 2023))]
workload_filtered_yearly_adjusted = pd.concat([workload_filtered_yearly_adjusted, mil_2023_adjusted], ignore_index=True)

# Calculate the average workload per surveyor per year for each location
average_workload_adjusted = workload_filtered_yearly_adjusted.groupby(['Location', 'Year'])['Workload'].mean().unstack()


average_workload_adjusted


Year,2021,2022,2023
Location,,,
cph,244.0,197.0,144.0
mil,58.0,125.0,91.0
par,67.0,82.0,142.0
